In [25]:
from fft_tdse_simulator.simulator import *
from fft_tdse_simulator.animator import *
import matplotlib.pyplot as plt
import numpy as np
matplotlib.use('AGG')


def potential(x, y):
    # Vmax = 4
    # return Vmax * (np.arctan(x/2) / np.pi + .5)
#    return -1.0 * np.exp(-.1*x*x)
    return -0.5 * (x*x + y*y + 0.25)**(-.5)

def init_wavefunction(x, sigma = 1.0, k0 = 2.0, x0 = -20.0):
    norm = 1.0/np.sqrt(sigma*np.sqrt(np.pi))
    return np.exp(-(x-x0)**2/(2*sigma) + 1j*k0*x) * norm

laser = LaserPulse(omega=0.25, t0 = 20, T = 80, E0 = 0.225)


sim1 = Simulator(verbose=True)
sim1.set_dimension(2)
#sim1.set_initial_condition(lambda x: init_wavefunction(x))
# sim1.set_mass(1.0)
# sim1.set_charge(-1.0)
sim1.set_potential(lambda x, y: potential(x,y))
sim1.set_laser_pulse(laser)
sim1.set_grid([-100, -100], [100, 100], [256, 256])
sim1.set_time_parameters(0.0, 100.0, 40000)
sim1.prepare()


ic| message: 'Dimension set to 2'
ic| 'Time grid:'
ic| self.t0: 0.0, self.t1: 100.0, self.n_steps: 40000


Initial guess: resid = 3.406251912231635, E = 5.390605530589333
Iteration 0, delta = 0.6547399401931608, resid = 2.076770213849125, E = 1.6373569994658679
Iteration 1, delta = 0.44450764643887347, resid = 0.7112276954357617, E = 0.47895201381778746
Iteration 2, delta = 0.2503010980009424, resid = 0.31865685627442586, E = 0.24298669701613201
Iteration 3, delta = 0.1688285299921537, resid = 0.1978473947401462, E = 0.15932042159445947
Iteration 4, delta = 0.1286532308837181, resid = 0.14448896640739844, E = 0.11610560014864607
Iteration 5, delta = 0.10511078155236156, resid = 0.1149573794507422, E = 0.08914183952321744
Iteration 6, delta = 0.09059253732847944, resid = 0.09724195428309437, E = 0.07009281809508863
Iteration 7, delta = 0.08259269363263508, resid = 0.08735799055827734, E = 0.054988750941804734
Iteration 8, delta = 0.08057226274602243, resid = 0.08410592244592507, E = 0.041314077666778415
Iteration 9, delta = 0.08466502763129588, resid = 0.08718116727514018, E = 0.026948742420

In [26]:
norm_hist = np.zeros_like(sim1.t_grid)
norm_hist[0] = sim1.wf.psiNorm()

#anim = Animator1d(sim1)
#anim.set_interval(80)

#anim_callback = anim.get_callback()

def callback(sim):
    norm_hist[sim.t_index+1] = sim.wf.psiNorm()
#    anim_callback(sim)

sim1.simulate(callback=callback)
#anim.make_movie("test.mp4")


ic| 'Running simulation...'


  0%|          | 0/40000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [27]:
import matplotlib.pyplot as plt

x,y = sim1.x, sim1.y
psi = sim1.psi
ic(psi.shape)


ic| psi.shape: (256, 256)


(256, 256)

In [28]:
from fft_tdse.psiviz import phase_mag_vis, mag_vis, dens_vis

def make_figure(simulator):
    # explain the next lines
    
    vis_type = 'complex'
    dens_factor = 0.75 # fraction of max density to show
    width = 1024
    height = 1024
    caption_format = 't = {sim.t:.2f}'
    
    fig_width = 10
    fig_height = 10
    fig, ax = plt.subplots(1, 1, figsize = (fig_width,fig_height))
    plt.axis('off')
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
    
    x_range = simulator.x
    y_range = simulator.y
    psi = simulator.psi
    xpix = len(x_range)
    ypix = len(y_range)
    
        
    if vis_type == 'density':
        max_dens = (np.abs(psi)**2).max() * dens_factor
    else:
        max_dens = np.abs(psi).max() * dens_factor

    if vis_type == 'complex':
        def mag_map(r):
            temp = r / max_dens

            temp = temp**.5

            return temp
        bmp = phase_mag_vis(psi.T, mag_map=mag_map)
    elif vis_type == 'magnitude':    
        def mag_map(r):
            temp = r / max_dens

            return temp
        bmp = mag_vis(psi.T, mag_map=mag_map)
    elif vis_type == 'density':
        def mag_map(r):
            temp = r / max_dens

            return temp
        bmp = dens_vis(psi.T, mag_map=mag_map)
    else:    
        raise ValueError(f"Unknown vis_type: {vis_type}")

    ax.clear()
    image = ax.imshow(bmp,origin='lower', extent = [np.min(x_range), np.max(x_range), np.min(y_range), np.max(x_range)])
    plt.savefig('test.png', dpi=width/fig_width)
    

    caption = caption_format.format(sim=simulator)
#        caption = f't = {t:.1f}, E_field = {Efield:.6f}, energy = {energy:.6f}'
    ax.text(.0125, .0125, caption, color = 'white', transform=ax.transAxes)
    
    plt.show()

In [29]:
make_figure(sim1)

/var/folders/qc/b2y8zg713_x554g8ykkcbcbh0000gp/T/ipykernel_20440/1322104197.py:61: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [18]:
test = '{sim.t:.2f}'
test.format(sim=sim1)

'100.00'

In [16]:
help(test.format)

Help on built-in function format:

format(...) method of builtins.str instance
    S.format(*args, **kwargs) -> str
    
    Return a formatted version of S, using substitutions from args and kwargs.
    The substitutions are identified by braces ('{' and '}').

